# Protein Folding | EPOCH Lab 2022

In [1]:
%matplotlib inline

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

import nglview as nv
import pytraj as pt

In [2]:
try:
  platform = Platform.getPlatformByName("CUDA")
except Exception:
  platform = Platform.getPlatformByName("OpenCL")

In [3]:
pdb = PDBFile('pdb/input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

In [4]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
modeller.topology

<Topology; 1 chains, 2798 residues, 8867 atoms, 6111 bonds>

In [5]:
system = forcefield.createSystem(modeller.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer,
                                 constraints=HBonds)

In [6]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

In [7]:
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('tmp/output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))

In [8]:
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-123888.38016541512,250.6780275708333
2000,-118179.72391541512,282.37615783903766
3000,-116293.28641541512,294.57622244399255
4000,-116109.09891541512,293.9801791821925
5000,-115624.75516541512,294.0307891944243
6000,-115522.94266541512,297.3024670675347
7000,-115266.28641541512,299.22106825934816
8000,-115805.66141541512,298.7152270769299
9000,-115763.31766541512,297.8488450437282
10000,-115554.34891541512,301.2164223241092


In [13]:
traj = pt.load('tmp/output.pdb')
view = nv.show_pytraj(traj)
view

NGLWidget(max_frame=9)